![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use Watsonx to respond to natural language questions using RAG approach

**Note:** Please note that for the watsonx challenge, please run these notebooks locally on your laptop/desktop and do not run it in IBM Cloud.  The instructions for running the notebook locally are provided in the readme.md file present in the zip file.

This notebook contains the steps and code to demonstrate support of Retrieval Augumented Generation in watsonx.ai. It introduces commands for data retrieval, knowledge base building & querying, and model testing.

Some familiarity with Python is helpful. This notebook uses Python 3.10.

#### About Retrieval Augmented Generation
Retrieval Augmented Generation (RAG) is a versatile pattern that can unlock a number of use cases requiring factual recall of information, such as querying a knowledge base in natural language.

In its simplest form, RAG requires 3 steps:

- Index knowledge base passages (once)
- Retrieve relevant passage(s) from knowledge base (for every user query)
- Generate a response by feeding retrieved passage into a large language model (for every user query)


<a id="setup"></a>
##  Set up the environment



### Install and import dependecies

In [818]:
!pip install chromadb==0.3.27
!pip install sentence_transformers
!pip install pandas
!pip install rouge_score
!pip install nltk
!pip install "ibm-watson-machine-learning>=1.0.312"

**Note:** Please restart the notebook kernel to pick up proper version of packages installed above.

In [819]:
import os, getpass
import pandas as pd
from typing import Optional, Dict, Any, Iterable, List

try:
    from sentence_transformers import SentenceTransformer
except ImportError:
    raise ImportError("Could not import sentence_transformers: Please install sentence-transformers package.")
    
try:
    import chromadb
    from chromadb.api.types import EmbeddingFunction
except ImportError:
    raise ImportError("Could not import chromdb: Please install chromadb package.")

### Watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [820]:
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "6BRv7jXYCVKwfMolURF89jgWWAQPZg7tur_EbGUWo6Uf"
}

### Defining the project id
The API requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

**Hint**: You can find the `project_id` as follows. Open the prompt lab in watsonx.ai. At the very top of the UI, there will be `Projects / <project name> /`. Click on the `<project name>` link. Then get the `project_id` from Project's Manage tab (Project -> Manage -> General -> Details).


In [821]:
try:
    project_id = "ba443f74-4a4e-4bad-a841-5b211ce0d7c8"
except KeyError:
    project_id = "ba443f74-4a4e-4bad-a841-5b211ce0d7c8"

<a id="data"></a>
## Train/test data loading

Load train and test datasets. At first, training dataset (`train_data`) should be used to work with the models to prepare and tune prompt. Then, test dataset (`test_data`) should be used to calculate the metrics score for selected model, defined prompts and parameters.

In [822]:
filename_test = 'Downloads/watsonx_challenge_evaluation_notebooks/data/RAG/nq910_400_instances/test.tsv'
filename_train = 'Downloads/watsonx_challenge_evaluation_notebooks/data/RAG/nq910_400_instances/train.tsv'

test_data_dict = pd.read_csv(filename_test, delimiter='\t')
train_data = pd.read_csv(filename_train, delimiter='\t')
test_data = pd.DataFrame(test_data_dict)
new_rows=[{"answers": "Redwood", "question": "Where was carter born?","qid":'123',"relevant":'267'} , 
         {"answers": "", "question": "which school did chris carter played basketball?","qid":'124',"relevant":'267'}]
new_data = pd.DataFrame(new_rows)
test_data = pd.concat([test_data, new_data], ignore_index=True)


In [823]:
train_data.head()

,qid,question,relevant,answers
0,8519,when was the cathedral of santa maria del fior...,136,begun in 1296::completed by 1436
1,852,who plays cassidy on law and order svu,177,Dean Winters
2,7600,who was the old woman in phantom of the opera,642,Madame Giry
3,4258,when is the finals of americas got talent 2017,781,"September 20 , 2017"
4,8272,who played the frog in gnomeo and juliet,1018,Ashley Jensen


In [824]:
test_data.head()

,qid,question,relevant,answers
0,5555,who did chris carter play for last year,267,Milwaukee Brewers
1,6654,what is the latest version of safari on mac,664,Safari 11
2,3396,when did bucharest become the capital of romania,944,1862
3,8198,who did jeffrey dean morgan play on supernatural,1398,John Eric Winchester
4,4526,who is the shortest man that ever lived,1522,Chandra Bahadur Dangi


## Build up knowledge base

The current state-of-the-art in RAG is to create dense vector representations of the knowledge base in order to calculate the semantic similarity to a given user query.

We can generate dense vector representations using embedding models. In this notebook, we use [SentenceTransformers](https://www.google.com/search?client=safari&rls=en&q=sentencetransformers&ie=UTF-8&oe=UTF-8) [all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) to embed both the knowledge base passages and user queries. `all-MiniLM-L6-v2` is a performant open-source model that is small enough to run locally.

A vector database is optimized for dense vector indexing and retrieval. This notebook uses [Chroma](https://docs.trychroma.com), a user-friendly open-source vector database, licensed under Apache 2.0, which offers good speed and performance with all-MiniLM-L6-v2 embedding model.

The dataset we are using is already split into self-contained passages that can be ingested by Chroma. 

The size of each passage is limited by the embedding model's context window (which is 256 tokens for `all-MiniLM-L6-v2`).

### Load knowledge base documents

Load set of documents used further to build knowledge base. 

In [825]:
data_root = "data"
knowledge_base_dir = f"{data_root}/knowledge_base"
print(knowledge_base_dir)

data/knowledge_base


In [826]:
if not os.path.exists(knowledge_base_dir):
    from zipfile import ZipFile
    with ZipFile(knowledge_base_dir + ".zip", 'r') as zObject:
        zObject.extractall(data_root)

In [827]:
documents = pd.read_csv("Downloads/knowledge_base/knowledge_base/psgs.tsv", sep='\t', header=0)
documents['indextext'] = documents['title'].astype(str) + "\n" + documents['text']

### Create an embedding function

Note that you can feed a custom embedding function to be used by chromadb. The performance of chromadb may differ depending on the embedding model used.

In [828]:
class MiniLML6V2EmbeddingFunction(EmbeddingFunction):
    MODEL = SentenceTransformer('all-MiniLM-L6-v2')
    def __call__(self, texts):
        return MiniLML6V2EmbeddingFunction.MODEL.encode(texts).tolist()
emb_func = MiniLML6V2EmbeddingFunction()

### Set up Chroma upsert

Upserting a document means update the document even if it exists in the database. Otherwise re-inserting a document throws an error. This is useful for experimentation purpose.

In [829]:
class ChromaWithUpsert:
    def __init__(
            self,
            name: Optional[str] = "watsonx_rag_collection",
            persist_directory:Optional[str]=None,
            embedding_function: Optional[EmbeddingFunction]=None,
            collection_metadata: Optional[Dict] = None,
    ):
        self._client_settings = chromadb.config.Settings()
        if persist_directory is not None:
            self._client_settings = chromadb.config.Settings(
                chroma_db_impl="duckdb+parquet",
                persist_directory=persist_directory,
            )
        self._client = chromadb.Client(self._client_settings)
        self._embedding_function = embedding_function
        self._persist_directory = persist_directory
        self._name = name
        self._collection = self._client.get_or_create_collection(
            name=self._name,
            embedding_function=self._embedding_function
            if self._embedding_function is not None
            else None,
            metadata=collection_metadata,
        )

    def upsert_texts(
        self,
        texts: Iterable[str],
        metadata: Optional[List[dict]] = None,
        ids: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> List[str]:
        """Run more texts through the embeddings and add to the vectorstore.
        Args:
            :param texts (Iterable[str]): Texts to add to the vectorstore.
            :param metadatas (Optional[List[dict]], optional): Optional list of metadatas.
            :param ids (Optional[List[str]], optional): Optional list of IDs.
            :param metadata: Optional[List[dict]] - optional metadata (such as title, etc.)
        Returns:
            List[str]: List of IDs of the added texts.
        """
        # TODO: Handle the case where the user doesn't provide ids on the Collection
        if ids is None:
            import uuid
            ids = [str(uuid.uuid1()) for _ in texts]
        embeddings = None
        self._collection.upsert(
            metadatas=metadata, documents=texts, ids=ids
        )
        return ids

    def is_empty(self):
        return self._collection.count()==0

    def persist(self):
        self._client.persist()

    def query(self, query_texts:str, n_results:int=5):
        """
        Returns the closests vector to the question vector
        :param query_texts: the question
        :param n_results: number of results to generate
        :return: the closest result to the given question
        """
        return self._collection.query(query_texts=query_texts, n_results=n_results)

### Embed and index documents with Chroma

**Note: Could take several minutes if you don't have pre-built indices**

In [830]:
%%time
chroma = ChromaWithUpsert(
    name=f"nq910_minilm6v2",
    embedding_function=emb_func,  # you can have something here using /embed endpoint
    persist_directory=knowledge_base_dir,
)
if chroma.is_empty():
    _ = chroma.upsert_texts(
        texts=documents.indextext.tolist(),
        # we handle tokenization, embedding, and indexing automatically. You can skip that and add your own embeddings as well
        metadata=[{'title': title, 'id': id}
                  for (title,id) in
                  zip(documents.title, documents.id)],  # filter on these!
        ids=[str(i) for i in documents.id],  # unique for each doc
    )
    chroma.persist()

CPU times: total: 500 ms
Wall time: 1.31 s


<a id="models"></a>
## Foundation Models on Watsonx

You need to specify `model_id` that will be used for inferencing.

**Action**: Use `FLAN_UL2` model.

In [831]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

In [832]:
model_id = ModelTypes.FLAN_UL2

<a id="predict"></a>
## Generate a retrieval-augmented response to a question

### Select questions

Get questions from the previously loaded test dataset.

In [833]:
question_texts = [q.strip("?") + "?" for q in test_data['question'].tolist()]
print("\n".join(question_texts))

who did chris carter play for last year?
what is the latest version of safari on mac?
when did bucharest become the capital of romania?
who did jeffrey dean morgan play on supernatural?
who is the shortest man that ever lived?
how many seconds do you have to throw a grenade?
who is known as a father of indian cricket?
what is the name of the period in japanese history that began in 1868?
harold and kumar go to white castle where was it filmed?
how many games have the capitals won in the playoffs?
what is the best selling nintendo game of all time?
korean movie about a man on an island?
who plays lorelai in hannah montana the movie?
what season of greys anatomy was the plane crash?
who made call of duty black ops 2?
when was the last solar eclipse seen in north america?
who is hosting the fifa world cup in 2022?
what is the record for wins in major league baseball?
big boss 2 telugu set location in hyderabad?
who did the us support in the korean war?
what is the caterpillar smoking in a

### Retrieve relevant context

Fetch paragraphs similar to the question.

In [834]:
relevant_contexts = []

for question_text in question_texts:
    relevant_chunks = chroma.query(
        query_texts=[question_text],
        n_results=6,
    )
    relevant_contexts.append(relevant_chunks)

Get the set of chunks for one of the questions.

In [835]:
sample_chunks = relevant_contexts[0]
for i, chunk in enumerate(sample_chunks['documents'][0]):
    print("=========")
    print("Paragraph index : ", sample_chunks['ids'][0][i])
    print("Paragraph : ", chunk)
    print("Distance : ", sample_chunks['distances'][0][i])

Paragraph index :  268
Paragraph :  Chris Carter (right-handed hitter)
2.2 Oakland Athletics 2.3 Houston Astros 2.4 Milwaukee Brewers 2.5 New York Yankees 2.6 Return to the Oakland Athletics 3 Personal life 4 References 5 External links Early life and career ( edit ) Carter was born in Redwood City , California . At approximately age 7 or 8 , his family moved to Las Vegas . He attended Sierra Vista High School . In 2005 , Sierra Vista 's baseball team won the Nevada Interscholastic Activities Association Class 4A state championship . Professional career ( edit ) Draft and minors ( edit ) Carter was drafted by the Chicago White Sox in the 15th round of the 2005 Major League Baseball Draft . Carter began his professional career with the Bristol White Sox of the Rookie - level Appalachian League in 2005 . He hit 10 home runs and had 37 runs batted in ( RBIs ) . He played for two teams in the 2006 season . The teams included the Great Falls White Sox of the Rookie - level Pioneer League an

### Feed the context and the questions to `watsonx.ai` model.

Define instructions for the model.

**Note:** Please **start with using [watsonx.ai Prompt Lab](https://dataplatform.cloud.ibm.com/wx/home?context=wx)** to find better prompts that provides you the best result on a small subset training records (under `train_data` variable). Make sure to not run an inference of all of `train_data`, as it'll take a long time to get the results. To get a sample from `train_data`, you can use e.g.`train_data.head(n=10)` to get first 10 records, or `train_data.sample(n=10)` to get random 10 records. Only once you have identified the best performing prompt, update this notebook to use the prompt and compute the metrics on the test data.

**Action:** Please edit the below cell and add your own prompt here. In the below prompt, we have the instruction (first sentence) and one example included in the prompt.  If you want to change the prompt or add your own examples or more examples, please change the below prompt accordingly.

In [867]:
def make_prompt(context, question_text):
    prompt = f"Given the provided context:\n{context}\n\nYour answer to the question:\n{question_text}"
    return prompt
prompt_texts = []
for context, question_text in zip(relevant_contexts, question_texts):
    prompt_text = make_prompt(context, question_text)
    prompt_texts.append(prompt_text)

Inspect prompt for sample question.

In [868]:
print(prompt_texts[1])

Given the provided context:
{'ids': [['649', '650', '648', '651', '661', '678']], 'embeddings': None, 'documents': [['Safari version history\n) Since 2016 macOS 10.13 High Sierra 11.0 ( September 19 , 2017 ) Since 2017 Microsoft Windows Windows 2000 3.0. 3 ( August 1 , 2007 ) Unofficial Windows XP SP1 4.0. 3 ( August 11 , 2009 ) 2007 -- 2009 Windows XP SP2 , SP3 5.1. 7 ( May 9 , 2012 ) 2007 -- 2012 Windows Vista Windows 7 2009 -- 2012 Release history ( edit ) Key : Old Beta Version Old Version Supported Version Current Version Current Beta Version Mac ( edit ) Major version Minor version WebKit version Operating System Release date Features Safari 0.8 48 Mac OS X v10. 2 January 7 , 2003 Public Beta . Initial release at Macworld San Francisco . 0.9 73 April 14 , 2003 Public Beta 2 . Tabbed browsing , forms and passwords autofill , browser reset ( removes cookies , cache , etc . ) , Netscape and Mozilla bookmarks importing , improved support for web standards , improved AppleScript suppo

### Defining the model parameters
We need to provide a set of model parameters that will influence the result:

In [974]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
parameters: {
  "decoding_method": "sample",
  "max_new_tokens": 20,
  "temperature": 0.2,
  "top_k": 100,
  "top_p": 0.9,
  "repetition_penalty": 1
}

Initialize the `Model` class.

In [975]:
from ibm_watson_machine_learning.foundation_models import Model

model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id)

### Generate a retrieval-augmented response

**Note:** Execution of this cell could take several minutes.

In [976]:
results = []

for prompt_text in prompt_texts:
    results.append(model.generate_text(prompt=prompt_text))

In [977]:
for idx, result in enumerate(results):
    print("Question = ", test_data.iloc[idx]['question'])
    print("Answer = ", result)
    print("Expected Answer(s) (may not be appear with exact wording in the dataset) = ", test_data.iloc[idx]['answers'])
    print("\n")

Question =  who did chris carter play for last year
Answer =  Milwaukee Brewers
Expected Answer(s) (may not be appear with exact wording in the dataset) =  Milwaukee Brewers


Question =  what is the latest version of safari on mac
Answer =  Safari 11
Expected Answer(s) (may not be appear with exact wording in the dataset) =  Safari 11


Question =  when did bucharest become the capital of romania
Answer =  1862
Expected Answer(s) (may not be appear with exact wording in the dataset) =  1862


Question =  who did jeffrey dean morgan play on supernatural
Answer =  John Winchester
Expected Answer(s) (may not be appear with exact wording in the dataset) =  John Eric Winchester


Question =  who is the shortest man that ever lived
Answer =  Chandra Bahadur Dangi
Expected Answer(s) (may not be appear with exact wording in the dataset) =  Chandra Bahadur Dangi


Question =  how many seconds do you have to throw a grenade
Answer =  4
Expected Answer(s) (may not be appear with exact wording in

<a id="score"></a>
## Calculate rougeL metric

In this sample notebook `rouge_score` module was used for rougeL calculation.

#### Rouge Metric

**Note:** The Rouge (Recall-Oriented Understudy for Gisting Evaluation) metric is a set of evaluation measures used in natural language processing (NLP) and specifically in text summarization and machine translation tasks. The Rouge metrics are designed to assess the quality of generated summaries or translations by comparing them to one or more reference texts.

The main idea behind Rouge is to measure the overlap between the generated summary (or translation) and the reference text(s) in terms of n-grams or longest common subsequences. By calculating recall, precision, and F1 scores based on these overlapping units, Rouge provides a quantitative assessment of the summary's content overlap with the reference(s).

Rouge-1 focuses on individual word overlap, Rouge-2 considers pairs of consecutive words, and Rouge-L takes into account the ordering of words and phrases. These metrics provide different perspectives on the similarity between two texts and can be used to evaluate different aspects of summarization or text generation models.

In [978]:
from rouge_score import rouge_scorer
from collections import defaultdict
import numpy as np

def get_rouge_score(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'])
    aggregate_score = defaultdict(list)

    for result, ref in zip(predictions, references):
        for key, val in scorer.score(result, ref).items():
            aggregate_score[key].append(val.fmeasure)

    scores = {}
    for key in aggregate_score:
        scores[key] = np.mean(aggregate_score[key])
    
    return scores

In [979]:
print(get_rouge_score(results, test_data.answers))

{'rouge1': 0.6270146520146519, 'rouge2': 0.32499999999999996, 'rougeL': 0.6270146520146519, 'rougeLsum': 0.6270146520146519}


---

Copyright © 2023 IBM. This notebook and its source code are released under the terms of the MIT License.